In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/TFM_2/utils')

In [ ]:
############
# MIT License
#
# Copyright (c) 2026 Rafael Asian, 2023 Minwoo Seong and 2022 MIT CSAIL and Joseph DelPreto
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.
#
# This file is a modified version originally written by Minwoo Seong.
# Source of the original version: https://github.com/dailyminiii/MultiSenseBadminton/blob/main/01_data_preprocessing_merge.py
# Date of modification: 2026.01.09
# Modified by: Rafael Asian
# See https://action-net.csail.mit.edu for more usage information.

In [ ]:
import h5py
import numpy as np
from scipy import interpolate  # for resampling
from scipy.signal import butter, lfilter  # for filtering
import os, glob
import pandas as pd
from collections import OrderedDict

In [ ]:
script_dir = '/content/drive/MyDrive/TFM_2'

In [ ]:
print(script_dir)

/content/drive/MyDrive/TFM_2


In [ ]:
import dict_utils

In [ ]:
#######################################
############ CONFIGURATION ############
#######################################

In [ ]:
# Define the modalities to use.
# Each entry is (device_name, stream_name, extraction_function)
# where extraction_function can select a subset of the stream columns.
device_streams_for_features = [
    ('eye-gaze', 'gaze', lambda data: data),
    ('gforce-lowerarm-emg', 'emg-values', lambda data: data),
    ('gforce-upperarm-emg', 'emg-values', lambda data: data),
    ('cgx-aim-leg-emg', 'emg-values', lambda data: data),
    ('moticon-insole', 'left-pressure', lambda data: data),
    ('moticon-insole', 'right-pressure', lambda data: data),
    ('moticon-insole', 'cop', lambda data: data),
    ('pns-joint', 'Euler-angle', lambda data: data),
]

In [ ]:
# Specify the input data.
data_root_dir = os.path.join(script_dir, 'data/Data_Archive')

In [ ]:
data_folders_bySubject = OrderedDict([
    ('Sub00', os.path.join(data_root_dir, 'Sub00')),
    ('Sub01', os.path.join(data_root_dir, 'Sub01')),
    ('Sub02', os.path.join(data_root_dir, 'Sub02')),
    ('Sub03', os.path.join(data_root_dir, 'Sub03')),
    ('Sub04', os.path.join(data_root_dir, 'Sub04')),
    # ('Sub05', os.path.join(data_root_dir, 'Sub05')),
    # ('Sub06', os.path.join(data_root_dir, 'Sub06')),
    ('Sub07', os.path.join(data_root_dir, 'Sub07')),
    # ('Sub08', os.path.join(data_root_dir, 'Sub08')),
    ('Sub09', os.path.join(data_root_dir, 'Sub09')),
    # ('Sub10', os.path.join(data_root_dir, 'Sub10')),
    ('Sub11', os.path.join(data_root_dir, 'Sub11')),
    # ('Sub12', os.path.join(data_root_dir, 'Sub12')),
    # ('Sub13', os.path.join(data_root_dir, 'Sub13')),
    ('Sub14', os.path.join(data_root_dir, 'Sub14')),
    ('Sub15', os.path.join(data_root_dir, 'Sub15')),
    # ('Sub16', os.path.join(data_root_dir, 'Sub16')),
    ('Sub17', os.path.join(data_root_dir, 'Sub17')),
    ('Sub18', os.path.join(data_root_dir, 'Sub18')),
    ('Sub19', os.path.join(data_root_dir, 'Sub19')),
    ('Sub20', os.path.join(data_root_dir, 'Sub20')),
    ('Sub21', os.path.join(data_root_dir, 'Sub21')),
    ('Sub22', os.path.join(data_root_dir, 'Sub22')),
    ('Sub23', os.path.join(data_root_dir, 'Sub23')),
    ('Sub24', os.path.join(data_root_dir, 'Sub24')),
])

In [ ]:
# Specify the labels to include.  These should match the labels in the HDF5 files.
baseline_label = 'None'
activities_to_classify = [  # Total Number is 3
    baseline_label,
    'Forehand Clear',
    'Backhand Driving',
]

In [ ]:
baseline_index = activities_to_classify.index(baseline_label)
# Some older experiments may have had different labels.
#  Each entry below maps the new name to a list of possible old names.
activities_renamed = {
    'Forehand Clear': ['Forehand Clear'], # Change name to Forehand clear
    'Backhand Driving': ['Backhand Driving'],
}

In [ ]:
# Define segmentation parameters.
resampled_Fs = 60  # define a resampling rate for all sensors to interpolate
num_segments_per_subject = 10
num_baseline_segments_per_subject = 10  # num_segments_per_subject*(max(1, len(activities_to_classify)-1))
segment_duration_s = 2.5
segment_length = int(round(resampled_Fs * segment_duration_s))
buffer_startActivity_s = 0.01
buffer_endActivity_s = 0.01

In [ ]:
# Define where outputs will be saved.
output_dir = os.path.join(script_dir, 'data_processed')
output_filepath = os.path.join(output_dir, 'data_processed_allStreams_60z_18subj_allActs_mulltiple_obj.hdf5') # output file name

# output_filepath = os.path.join(output_dir, 'data_processed_allStreams_100hz_18subj_allActs_skill_level.hdf5') # output file name
annotation_data_filePath = '/content/drive/MyDrive/TFM_2/data/Documentations/Annotation Data File.xlsx' # directory of annotation data xlsx file
# output_filepath = None

In [ ]:
# Define filtering parameters.
filter_cutoff_emg_Hz = 5
filter_cutoff_emg_cognionics_Hz = 20
filter_cutoff_pressure_Hz = 5
filter_cutoff_gaze_Hz = 5

In [ ]:
# Make the output folder if needed.
if output_dir is not None:
    os.makedirs(output_dir, exist_ok=True)
    print('\n')
    print('Saving outputs to')
    print(output_filepath)
    print('\n')



Saving outputs to
/content/drive/MyDrive/TFM_2/data_processed/data_processed_allStreams_60z_18subj_allActs_mulltiple_obj.hdf5




In [ ]:
################################################
############ INTERPOLATE AND FILTER ############
################################################

In [ ]:
# Will filter each column of the data.
def lowpass_filter(data, cutoff, Fs, order=4):
    nyq = 0.5 * Fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data.T).T
    return y

In [ ]:
def convert_to_nan(arr, difff, time):
    for i in range(len(arr) - time):
        for j in range(len(arr[0])):
            diff = abs(arr[i, j] - arr[i + time, j])
            if diff > difff:
                arr[i, j] = np.nan
    return arr

In [ ]:
# Load the original data.
data_bySubject = {}
for (subject_id, data_folder) in data_folders_bySubject.items():
    print()
    print('id : ', subject_id)
    print()
    print('Loading data for subject %s' % subject_id)
    data_bySubject[subject_id] = []
    hdf_filepaths = glob.glob(os.path.join(data_folder, '**/*.hdf5'), recursive=True)
    print(hdf_filepaths)
    for hdf_filepath in hdf_filepaths:
        data_bySubject[subject_id].append({})
        hdf_file = h5py.File(hdf_filepath, 'r')
        # Add the activity label information.
        have_all_streams = True
        # try:
        #     device_name = 'experiment-activities'
        #     stream_name = 'activities'
        #     data_bySubject[subject_id][-1].setdefault(device_name, {})
        #     data_bySubject[subject_id][-1][device_name].setdefault(stream_name, {})
        #     for key in ['time_s', 'data']:
        #         data_bySubject[subject_id][-1][device_name][stream_name][key] = hdf_file[device_name][stream_name][key][
        #                                                                         :]
        #     num_activity_entries = len(data_bySubject[subject_id][-1][device_name][stream_name]['time_s'])
        #     if num_activity_entries == 0:
        #         have_all_streams = False
        #     elif data_bySubject[subject_id][-1][device_name][stream_name]['time_s'][0] == 0:
        #         have_all_streams = False
        # except KeyError:
        #     have_all_streams = False
        # Load data for each of the streams that will be used as features.
        for (device_name, stream_name, _) in device_streams_for_features:
            data_bySubject[subject_id][-1].setdefault(device_name, {})
            data_bySubject[subject_id][-1][device_name].setdefault(stream_name, {})
            for key in ['time_s', 'data']:
                try:
                    data_bySubject[subject_id][-1][device_name][stream_name][key] = hdf_file[device_name][stream_name][
                                                                                        key][:]
                except KeyError:
                    have_all_streams = False
        if not have_all_streams:
            data_bySubject[subject_id].pop()
            print('  Ignoring HDF5 file:', hdf_filepath)
        hdf_file.close()

print(data_bySubject)


id :  Sub00

Loading data for subject Sub00
['/content/drive/MyDrive/TFM_2/data/Data_Archive/Sub00/2023-01-02_10-54-08_streamLog_badminton-wearables_Sub00.hdf5', '/content/drive/MyDrive/TFM_2/data/Data_Archive/Sub00/2023-01-02_11-30-40_streamLog_badminton-wearables_Sub00.hdf5']

id :  Sub01

Loading data for subject Sub01
['/content/drive/MyDrive/TFM_2/data/Data_Archive/Sub01/2023-01-31_11-08-03_streamLog_badminton-wearables_Sub01.hdf5', '/content/drive/MyDrive/TFM_2/data/Data_Archive/Sub01/2023-01-31_11-49-52_streamLog_badminton-wearables_Sub01.hdf5']

id :  Sub02

Loading data for subject Sub02
['/content/drive/MyDrive/TFM_2/data/Data_Archive/Sub02/2023-02-16_10-20-47_streamLog_badminton-wearables_Sub02.hdf5', '/content/drive/MyDrive/TFM_2/data/Data_Archive/Sub02/2023-02-16_11-04-52_streamLog_badminton-wearables_Sub02.hdf5']

id :  Sub03

Loading data for subject Sub03
['/content/drive/MyDrive/TFM_2/data/Data_Archive/Sub03/2023-01-04_10-25-48_streamLog_badminton-wearables_Sub03.hdf5

In [ ]:
# Filter data.
print()
for (subject_id, file_datas) in data_bySubject.items():
    print('Filtering data for subject %s' % subject_id)
    for (data_file_index, file_data) in enumerate(file_datas):
        # print('file_data : ', file_data)
        print(' Data file index', data_file_index)
        # Filter EMG data.
        for gforce_key in ['gforce-lowerarm-emg', 'gforce-upperarm-emg']:
            if gforce_key in file_data:
                t = file_data[gforce_key]['emg-values']['time_s']
                Fs = (t.size - 1) / (t[-1] - t[0])
                print(' Filtering %s with Fs %0.1f Hz to cutoff %f' % (gforce_key, Fs, filter_cutoff_emg_Hz))
                data_stream = file_data[gforce_key]['emg-values']['data'][:, :]
                y = np.abs(data_stream)
                y = lowpass_filter(y, filter_cutoff_emg_Hz, Fs)
                # for i in range(len(data_stream[0])):
                #     plt.plot(t-t[0], data_stream[:, i], label=gforce_key+'_raw')
                #     plt.plot(t-t[0], y[:, i], label=gforce_key+'_preprocessed')
                #     plt.legend()
                #     plt.show()
                #     plt.clf()
                #     plt.plot(t[500:900] - t[0], data_stream[500:900, i], label=gforce_key + '_raw')
                #     plt.plot(t[500:900] - t[0], y[500:900, i], label=gforce_key + '_preprocessed')
                #     plt.legend()
                #
                #     plt.show()
                #     plt.clf()
                file_data[gforce_key]['emg-values']['data'] = y
        for cognionics_key in ['cgx-aim-leg-emg']:
            if cognionics_key in file_data:
                t = file_data[cognionics_key]['emg-values']['time_s']
                Fs = (t.size - 1) / (t[-1] - t[0])
                print(' Filtering %s with Fs %0.1f Hz to cutoff %f' % (
                cognionics_key, Fs, filter_cutoff_emg_cognionics_Hz))
                data_stream = file_data[cognionics_key]['emg-values']['data'][:, :]
                data_stream = np.abs(data_stream)
                # Correcting the bounce value
                y = convert_to_nan(data_stream, difff=80, time=5)
                y[y > 26000] = np.nan
                # y[y < -26000] = np.nan
                # y[y < -26000] = np.nan
                df = pd.DataFrame(y)
                # print(df.isnull().sum())
                for ii in range(len(df.loc[0])):
                    df.loc[:, ii] = df.loc[:, ii].fillna(df.loc[:, ii].median())
                    # print(df.loc[:, ii].mean())
                # print(df.isnull().sum())
                y = df.to_numpy()
                y = lowpass_filter(y, filter_cutoff_emg_cognionics_Hz, Fs)
                # for i in range(len(data_stream[0])):
                #     # print('max', np.amax(data_stream[:, i]))
                #     # print('min', np.amin(data_stream[:, i]))
                #     plt.plot(t-t[0], data_stream[:, i], label=cognionics_key+'_raw_channel' + str(i+1))
                #     plt.plot(t - t[0], y[:, i], label=cognionics_key + '_preprocessed_channel'+ str(i+1))
                #     plt.legend()
                #     plt.show()
                #     plt.clf()
                #     plt.plot(t[50000:55000] - t[0], data_stream[50000:55000, i], label=cognionics_key + '_raw_channel' + str(i+1))
                #     plt.plot(t[50000:55000] - t[0], y[50000:55000, i], label=cognionics_key + '_preprocessed_channel' + str(i+1))
                #     plt.legend()
                #     plt.show()
                #     plt.clf()
                #     plt.plot(t[50000:55000] - t[0], y[50000:55000, i], label=cognionics_key + '_preprocessed_channel' + str(i+1))
                #     plt.legend()
                #     plt.show()
                #     plt.clf()
                file_data[cognionics_key]['emg-values']['data'] = y
        # Filter eye-gaze data.
        if 'eye-gaze' in file_data:
            t = file_data['eye-gaze']['gaze']['time_s']
            Fs = (t.size - 1) / (t[-1] - t[0])
            data_stream = file_data['eye-gaze']['gaze']['data'][:, :]
            y = data_stream
            # # Apply a ZOH to remove clipped values.
            # #  The gaze position is already normalized to video coordinates,
            # #   so anything outside [0,1] is outside the video.
            # print(' Holding clipped values in %s' % ('eye-gaze'))
            clip_low_x = 0 + 0.05
            clip_high_x = 1088 - 0.05
            clip_low_y = 0 + 0.05
            clip_high_y = 1080 - 0.05
            y[:, 0] = np.clip(y[:, 0], clip_low_x, clip_high_x)
            y[:, 1] = np.clip(y[:, 1], clip_low_y, clip_high_y)
            y[y == clip_low_x] = np.nan
            y[y == clip_high_x] = np.nan
            y[y == clip_low_y] = np.nan
            y[y == clip_high_y] = np.nan
            y = pd.DataFrame(y).interpolate(method='zero').to_numpy()
            # # Replace any remaining NaNs with a dummy value,
            # #  in case the first or last timestep was clipped (interpolate() does not extrapolate).
            y[np.isnan(y)] = 540
            # plt.plot(t-t[0], y[:,0], '*-')
            # plt.ylim(-2,2)
            # Filter to smooth.
            print('   Filtering %s with Fs %0.1f Hz to cutoff %f' % ('eye-gaze', Fs, filter_cutoff_gaze_Hz))
            y = lowpass_filter(y, filter_cutoff_gaze_Hz, Fs)
            # for i in range(len(data_stream[0])):
            #     plt.plot(t - t[0], data_stream[:, i], label='eye-gaze' + '_raw')
            #     plt.plot(t-t[0], y[:, i], label='eye-gaze'+'_preprocessed')
            #     plt.legend()
            #     plt.show()
            #     plt.clf()
            file_data['eye-gaze']['gaze']['data'] = y
        for moticon_key in ['left-pressure', 'right-pressure', 'cop']:
            if moticon_key in file_data['moticon-insole']:
                t = file_data['moticon-insole'][moticon_key]['time_s']
                Fs = (t.size - 1) / (t[-1] - t[0])
                print(' Filtering %s with Fs %0.1f Hz to cutoff %f' % ('moticon-insole', Fs, filter_cutoff_pressure_Hz))
                data_stream = file_data['moticon-insole'][moticon_key]['data'][:, :]
                y = np.abs(data_stream)
                y = lowpass_filter(y, filter_cutoff_pressure_Hz, Fs)
                # plt.plot(t-t[0], data_stream[:,0], label=moticon_key+'_raw')
                # plt.plot(t-t[0], y[:,0], label=moticon_key+'_preprocessed')
                # plt.legend()
                # plt.show()
                file_data['moticon-insole'][moticon_key]['data'] = y
        data_bySubject[subject_id][data_file_index] = file_data


Filtering data for subject Sub00
 Data file index 0
 Filtering gforce-lowerarm-emg with Fs 76.5 Hz to cutoff 5.000000
 Filtering gforce-upperarm-emg with Fs 162.2 Hz to cutoff 5.000000
 Filtering cgx-aim-leg-emg with Fs 502.3 Hz to cutoff 20.000000


/tmp/ipython-input-222663362.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(' Filtering %s with Fs %0.1f Hz to cutoff %f' % (gforce_key, Fs, filter_cutoff_emg_Hz))
/tmp/ipython-input-222663362.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(' Filtering %s with Fs %0.1f Hz to cutoff %f' % (
/tmp/ipython-input-222663362.py:96: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('   Filtering %s with Fs %0.1f Hz to cutoff %f' % ('eye-gaze', Fs, 

   Filtering eye-gaze with Fs 30.0 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 53.3 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 53.3 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 53.3 Hz to cutoff 5.000000
 Data file index 1
 Filtering gforce-lowerarm-emg with Fs 96.4 Hz to cutoff 5.000000
 Filtering gforce-upperarm-emg with Fs 171.5 Hz to cutoff 5.000000
 Filtering cgx-aim-leg-emg with Fs 502.2 Hz to cutoff 20.000000


/tmp/ipython-input-222663362.py:96: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('   Filtering %s with Fs %0.1f Hz to cutoff %f' % ('eye-gaze', Fs, filter_cutoff_gaze_Hz))
/tmp/ipython-input-222663362.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(' Filtering %s with Fs %0.1f Hz to cutoff %f' % ('moticon-insole', Fs, filter_cutoff_pressure_Hz))
/tmp/ipython-input-222663362.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(' Filtering %s

   Filtering eye-gaze with Fs 29.5 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 52.5 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 52.5 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 52.5 Hz to cutoff 5.000000
Filtering data for subject Sub01
 Data file index 0
 Filtering gforce-lowerarm-emg with Fs 94.2 Hz to cutoff 5.000000
 Filtering gforce-upperarm-emg with Fs 137.8 Hz to cutoff 5.000000
 Filtering cgx-aim-leg-emg with Fs 502.0 Hz to cutoff 20.000000


...

/tmp/ipython-input-222663362.py:96: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('   Filtering %s with Fs %0.1f Hz to cutoff %f' % ('eye-gaze', Fs, filter_cutoff_gaze_Hz))
/tmp/ipython-input-222663362.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(' Filtering %s with Fs %0.1f Hz to cutoff %f' % ('moticon-insole', Fs, filter_cutoff_pressure_Hz))
/tmp/ipython-input-222663362.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(' Filtering %s

   Filtering eye-gaze with Fs 30.0 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 59.6 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 59.6 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 59.6 Hz to cutoff 5.000000
Filtering data for subject Sub24
 Data file index 0
 Filtering gforce-lowerarm-emg with Fs 98.4 Hz to cutoff 5.000000
 Filtering gforce-upperarm-emg with Fs 89.7 Hz to cutoff 5.000000
 Filtering cgx-aim-leg-emg with Fs 502.1 Hz to cutoff 20.000000


/tmp/ipython-input-222663362.py:96: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('   Filtering %s with Fs %0.1f Hz to cutoff %f' % ('eye-gaze', Fs, filter_cutoff_gaze_Hz))
/tmp/ipython-input-222663362.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(' Filtering %s with Fs %0.1f Hz to cutoff %f' % ('moticon-insole', Fs, filter_cutoff_pressure_Hz))
/tmp/ipython-input-222663362.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(' Filtering %s

   Filtering eye-gaze with Fs 30.0 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 49.9 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 49.9 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 49.9 Hz to cutoff 5.000000
 Data file index 1
 Filtering gforce-lowerarm-emg with Fs 98.7 Hz to cutoff 5.000000
 Filtering gforce-upperarm-emg with Fs 89.8 Hz to cutoff 5.000000
 Filtering cgx-aim-leg-emg with Fs 502.0 Hz to cutoff 20.000000
   Filtering eye-gaze with Fs 30.0 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 52.7 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 52.7 Hz to cutoff 5.000000
 Filtering moticon-insole with Fs 52.7 Hz to cutoff 5.000000


/tmp/ipython-input-222663362.py:96: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print('   Filtering %s with Fs %0.1f Hz to cutoff %f' % ('eye-gaze', Fs, filter_cutoff_gaze_Hz))
/tmp/ipython-input-222663362.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(' Filtering %s with Fs %0.1f Hz to cutoff %f' % ('moticon-insole', Fs, filter_cutoff_pressure_Hz))


In [ ]:
print()
for (subject_id, file_datas) in data_bySubject.items():
    print('Normalizing data for subject %s' % subject_id)
    for (data_file_index, file_data) in enumerate(file_datas):
        # Normalize gForce Pro EMG data.
        for gforce_key in ['gforce-lowerarm-emg', 'gforce-upperarm-emg']:
            if gforce_key in file_data:
                data_stream = file_data[gforce_key]['emg-values']['data'][:, :]
                min_val = 0
                max_val = 300
                y = data_stream
                print(' Normalizing %s with min/max [%0.1f, %0.1f]' % (gforce_key, np.amin(y), np.amax(y)))
                # Normalize them jointly.
                y = y / ((max_val - min_val) / 2)
                # Jointly shift the baseline to -1 instead of 0.
                y = y - np.amin(y) - 1
                file_data[gforce_key]['emg-values']['data'] = y
                print('   Now has range [%0.1f, %0.1f]' % (np.amin(y), np.amax(y)))
                # plt.plot(y.reshape(y.shape[0], -1))
                # plt.show()

        # Normalize Cognionics EMG data.
        for cognionics_key in ['cgx-aim-leg-emg']:
            if cognionics_key in file_data:
                data_stream = file_data[cognionics_key]['emg-values']['data'][:, :]
                y = data_stream
                print(' Normalizing %s with min/max [%0.1f, %0.1f]' % (cognionics_key, np.amin(y), np.amax(y)))
                y = y / ((np.amax(y) - np.amin(y)) / 2)
                # Jointly shift the baseline to -1 instead of 0.
                y = y - np.amin(y) - 1
                file_data[cognionics_key]['emg-values']['data'] = y
                print('   Now has range [%0.1f, %0.1f]' % (np.amin(y), np.amax(y)))
                # plt.plot(y.reshape(y.shape[0], -1))
                # plt.show()

        # Normalize Perception Neuron Studio joints.
        if 'pns-joint' in file_data:
            data_stream = file_data['pns-joint']['Euler-angle']['data'][:, :]
            y = data_stream
            min_val = -180
            max_val = 180
            print(' Normalizing %s with forced min/max [%0.1f, %0.1f]' % ('pns-joint-euler', min_val, max_val))
            # Normalize all at once since using fixed bounds anyway.
            # Preserve relative bends, such as left arm being bent more than the right.
            y = y / ((max_val - min_val) / 2)
            file_data['pns-joint']['Euler-angle']['data'] = y
            print('   Now has range [%0.1f, %0.1f]' % (np.amin(y), np.amax(y)))
            # plt.plot(y.reshape(y.shape[0], -1))
            # plt.show()

        # Normalize eyetracking gaze.
        if 'eye-gaze' in file_data:
            data_stream = file_data['eye-gaze']['gaze']['data'][:]
            t = file_data['eye-gaze']['gaze']['time_s'][:]
            y = data_stream
            min_x = 0
            max_x = 1088
            min_y = 0
            max_y = 1080

            print(' Normalizing %s with min/max [%0.1f, %0.1f] and min/max [%0.1f, %0.1f]' % (
            'eye-gaze', min_x, max_x, min_y, max_y))
            # # The gaze position is already normalized to video coordinates,
            # #  so anything outside [0,1] is outside the video.
            clip_low = -0.95
            clip_high = 0.95

            # y = np.clip(y, clip_low, clip_high)
            # Put in range [-1, 1] for extra resolution.
            # Normalize them jointly.
            y[:, 0] = y[:, 0] / ((max_x - min_x) / 2)
            y[:, 1] = y[:, 1] / ((max_y - min_y) / 2)
            # Jointly shift the baseline to -1 instead of 0.
            y = y - min_y - 1
            # y = (y - np.mean([clip_low, clip_high])) / ((clip_high - clip_low) / 2)
            # print(' Clipping %s to [%0.1f, %0.1f]' % ('eye-gaze', clip_low, clip_high))
            # plt.plot(t-t[0], y)
            # plt.show()
            file_data['eye-gaze']['gaze']['data'] = y
            print('   Now has range [%0.1f, %0.1f]' % (np.amin(y), np.amax(y)))
            # plt.plot(y.reshape(y.shape[0], -1))
            # plt.show()

        # Normalize Moticon Pressure.
        for moticon_key in ['left-pressure', 'right-pressure', 'cop']:
            if 'moticon-insole' in file_data:
                data_stream = file_data['moticon-insole'][moticon_key]['data'][:, :]
                y = data_stream
                print(' Normalizing %s with min/max [%0.1f, %0.1f]' % ('moticon-insole', np.amin(y), np.amax(y)))
                # Normalize them jointly.
                y = y / ((np.amax(y) - np.amin(y)) / 2)
                # Jointly shift the baseline to -1 instead of 0.
                y = y - np.amin(y) - 1
                file_data['moticon-insole'][moticon_key]['data'] = y
                print('   Now has range [%0.1f, %0.1f]' % (np.amin(y), np.amax(y)))
                # plt.plot(y.reshape(y.shape[0], -1))
                # plt.show()

        data_bySubject[subject_id][data_file_index] = file_data


Normalizing data for subject Sub00
 Normalizing gforce-lowerarm-emg with min/max [0.1, 241.2]
   Now has range [-1.0, 0.6]
 Normalizing gforce-upperarm-emg with min/max [0.0, 133.2]
   Now has range [-1.0, -0.1]
 Normalizing cgx-aim-leg-emg with min/max [0.3, 19829.4]
   Now has range [-1.0, 1.0]
 Normalizing pns-joint-euler with forced min/max [-180.0, 180.0]
   Now has range [-1.0, 1.0]
 Normalizing eye-gaze with min/max [0.0, 1088.0] and min/max [0.0, 1080.0]
   Now has range [-1.1, 0.9]
 Normalizing moticon-insole with min/max [-1.9, 23.3]
   Now has range [-1.0, 1.0]
 Normalizing moticon-insole with min/max [-1.5, 20.1]
   Now has range [-1.0, 1.0]
 Normalizing moticon-insole with min/max [-0.0, 0.4]
   Now has range [-1.0, 1.0]
 Normalizing gforce-lowerarm-emg with min/max [0.1, 217.2]
   Now has range [-1.0, 0.4]
 Normalizing gforce-upperarm-emg with min/max [0.0, 223.3]
   Now has range [-1.0, 0.5]
 Normalizing cgx-aim-leg-emg with min/max [-2142.7, 24506.8]
   Now has range [

In [ ]:
# Aggregate data (and normalize if needed).
print()
for (subject_id, file_datas) in data_bySubject.items():
    print('Aggregating data for subject %s' % subject_id)
    for (data_file_index, file_data) in enumerate(file_datas):
        # Aggregate EMG data.
        for gforce_key in ['gforce-lowerarm-emg', 'gforce-upperarm-emg']:
            if gforce_key in file_data:
                pass

        # Aggregate eye-tracking gaze.
        if 'cgx-aim-leg-emg' in file_data:
            pass

        # Aggregate Perception Nueron Studio joints.
        if 'pns-joint' in file_data:
            pass

        # Aggregate eye-tracking gaze.
        if 'eye-gaze' in file_data:
            pass

        # Aggregate eye-tracking gaze.
        if 'moticon-insole' in file_data:
            pass

        data_bySubject[subject_id][data_file_index] = file_data


Aggregating data for subject Sub00
Aggregating data for subject Sub01
Aggregating data for subject Sub02
Aggregating data for subject Sub03
Aggregating data for subject Sub04
Aggregating data for subject Sub07
Aggregating data for subject Sub09
Aggregating data for subject Sub11
Aggregating data for subject Sub14
Aggregating data for subject Sub15
Aggregating data for subject Sub17
Aggregating data for subject Sub18
Aggregating data for subject Sub19
Aggregating data for subject Sub20
Aggregating data for subject Sub21
Aggregating data for subject Sub22
Aggregating data for subject Sub23
Aggregating data for subject Sub24


In [ ]:
example_labels = []
example_label_indexes = []
example_matrices_list = []
example_subject_ids = []
example_skill_level = []
example_score_annot_3_hori = []
example_score_annot_3_ver = []
example_score_annot_4 = []
example_score_annot_5 = []

In [ ]:
Forehand_time_list = []
Backhand_time_list = []
NoActivity_time_list = []

In [ ]:
df = pd.read_excel(annotation_data_filePath)

In [ ]:
print(df)

     Subject Number  Annotation Start Time  Annotation Stop Time  Stroke Num  \
0             Sub00           1.672625e+09          1.672625e+09           1   
1             Sub00           1.672625e+09          1.672625e+09           2   
2             Sub00           1.672625e+09          1.672625e+09           3   
3             Sub00           1.672625e+09          1.672625e+09           4   
4             Sub00           1.672625e+09          1.672625e+09           5   
...             ...                    ...                   ...         ...   
7758          Sub24           1.676441e+09          1.676441e+09         150   
7759          Sub24           1.676441e+09          1.676441e+09         151   
7760          Sub24           1.676441e+09          1.676441e+09         152   
7761          Sub24           1.676441e+09          1.676441e+09         153   
7762          Sub24           1.676441e+09          1.676441e+09         154   

     Annotation Level 1\n(Stroke Type) 

In [ ]:
print(len(df))

7763


In [ ]:
# Remove rows where the specified columns contain "NoVid"
df_filtered = df[
    (df["Annotation Level 3\n(Landing Location - Horizontal)"] != "NoVid") &
    (df["Annotation Level 3\n(Landing Location - Vertical)"] != "NoVid") &
    (df["Annotation Level 4\n(Hitting Location, major voting)"] != "NoVid") &
    (df["Annotation Level 5\n(Hitting Sound, major voting)"] != "NoVid")
    ]

In [ ]:
print(len(df_filtered))

6906


In [ ]:
print(set(df_filtered['Annotation Level 2\n(Skill Level)'] ))

{'Intermediate', 'Beginner', 'Expert'}


In [ ]:
df_filtered['Annotation Level 2\n(Skill Level)'] = df_filtered['Annotation Level 2\n(Skill Level)'].map({
    'Beginner': 0,
    'Intermediate': 1,
    'Expert': 2
})

/tmp/ipython-input-924513887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Annotation Level 2\n(Skill Level)'] = df_filtered['Annotation Level 2\n(Skill Level)'].map({


In [ ]:
print(set(df_filtered['Annotation Level 2\n(Skill Level)'] ))

{0, 1, 2}


In [ ]:
print(df_filtered['Annotation Level 2\n(Skill Level)'])

0       0
1       0
2       0
3       0
4       0
       ..
7758    2
7759    2
7760    2
7761    2
7762    2
Name: Annotation Level 2\n(Skill Level), Length: 6906, dtype: int64


In [ ]:
for (subject_id, file_datas) in data_bySubject.items():
    print('Resampling data for subject %s' % subject_id)

    df_subject = df_filtered[df_filtered["Subject Number"] == subject_id]

    df_subject_forehand = df_subject[df_subject["Annotation Level 1\n(Stroke Type)"] == 'Forehand Clear']
    df_subject_backhand = df_subject[df_subject["Annotation Level 1\n(Stroke Type)"] == 'Backhand Driving']

    for (data_file_index, file_data) in enumerate(file_datas):

        sub_example_labels = []
        sub_example_label_indexes = []
        sub_example_subject_ids = []
        sub_example_skill_level = []

        sub_example_score_annot_3_hori = []
        sub_example_score_annot_3_ver = []
        sub_example_score_annot_4 = []
        sub_example_score_annot_5 = []

        Forehand_start_time_list = df_subject_forehand['Annotation Start Time'].values.tolist()
        Forehand_stop_time_list = df_subject_forehand['Annotation Stop Time'].values.tolist()
        Backhand_start_time_list = df_subject_backhand['Annotation Start Time'].values.tolist()
        Backhand_stop_time_list = df_subject_backhand['Annotation Stop Time'].values.tolist()

        Forehand_skill_level_list = df_subject_forehand['Annotation Level 2\n(Skill Level)'].values.tolist()
        Backhand_skill_level_list = df_subject_backhand['Annotation Level 2\n(Skill Level)'].values.tolist()

        Forehand_score_annot_3_hori_list = df_subject_forehand["Annotation Level 3\n(Landing Location - Horizontal)"].values.tolist()
        Backhand_score_annot_3_hori_list = df_subject_backhand["Annotation Level 3\n(Landing Location - Horizontal)"].values.tolist()
        Forehand_score_annot_3_ver_list = df_subject_forehand["Annotation Level 3\n(Landing Location - Vertical)"].values.tolist()
        Backhand_score_annot_3_ver_list = df_subject_backhand["Annotation Level 3\n(Landing Location - Vertical)"].values.tolist()

        Forehand_score_annot_4_list = df_subject_forehand["Annotation Level 4\n(Hitting Location, major voting)"].values.tolist()
        Backhand_score_annot_4_list = df_subject_backhand["Annotation Level 4\n(Hitting Location, major voting)"].values.tolist()
        Forehand_score_annot_5_list = df_subject_forehand["Annotation Level 5\n(Hitting Sound, major voting)"].values.tolist()
        Backhand_score_annot_5_list = df_subject_backhand["Annotation Level 5\n(Hitting Sound, major voting)"].values.tolist()

        NoActivity_start_time_list = []
        NoActivity_stop_time_list = []
        NoActivity_start_time_list.extend(Forehand_stop_time_list[0:-1])
        NoActivity_start_time_list.extend(Backhand_stop_time_list[0:-1])
        NoActivity_stop_time_list.extend(Forehand_start_time_list[1:])
        NoActivity_stop_time_list.extend(Backhand_start_time_list[1:])

        example_matrices_device_eye_gaze = []
        example_matrices_device_gforce_lowerarm_emg = []
        example_matrices_device_gforce_upperarm_emg = []
        example_matrices_device_cgx_aim_emg = []
        example_matrices_device_moticon_insole_left_pressure = []
        example_matrices_device_moticon_insole_right_pressure = []
        example_matrices_device_moticon_insole_cop = []
        example_matrices_device_pns_joint_Euler = []

        print('Total Labeling Number :', len(Forehand_start_time_list) + len(Backhand_start_time_list) + len(NoActivity_start_time_list))
        print('Saved Highclear Labeling Number :', len(Forehand_start_time_list), len(Forehand_stop_time_list))
        print('Saved Backhand Labeling Number :', len(Backhand_start_time_list), len(Backhand_stop_time_list))
        print('Saved NoActivity Labeling Number :', len(NoActivity_start_time_list), len(NoActivity_stop_time_list))

        print('Save Forehand Score annot 3 Hori Number :', len(Forehand_score_annot_3_hori_list))
        print('Save Forehand Score annot 3 Ver Number :', len(Forehand_score_annot_3_ver_list))
        print('Save Forehand Score annot 4 Number :', len(Forehand_score_annot_4_list))
        print('Save Forehand Score annot 5 Number :', len(Forehand_score_annot_5_list))

        # print('Save Backhand Score Number :', len(Backhand_score_list))
        print('Save Backhand Score annot 3 Hori Number :', len(Backhand_score_annot_3_hori_list))
        print('Save Backhand Score annot 3 Ver Number :', len(Backhand_score_annot_3_ver_list))
        print('Save Backhand Score annot 4 Number :', len(Backhand_score_annot_4_list))
        print('Save Backhand Score annot 5 Number :', len(Backhand_score_annot_5_list))


        device_num = 1
        for (device_name, stream_name, _) in device_streams_for_features:
            example_matrices_each_file = []
            print("Device Name :", device_name)
            data = np.squeeze(np.array(file_data[device_name][stream_name]['data']))
            time_s = np.squeeze(np.array(file_data[device_name][stream_name]['time_s']))

            _, unique_indices = np.unique(time_s, return_index=True)
            time_s = time_s[unique_indices]
            data = data[unique_indices]

            label_indexes = [0] * len(time_s)

            # Initialize the Number of each stroke
            Num_base = 0
            Num_clear = 0
            Num_drive = 0

            # Save the Forehand Clear Data
            highNum = 0
            backNum = 0
            baseNum = 0

            for j in range(len(Forehand_start_time_list)):
                # Save the swing time of each stroke
                Forehand_time_list.append(Forehand_stop_time_list[j]-Forehand_start_time_list[j])
                # time indexing
                high_time_indexes = np.where((time_s >= Forehand_start_time_list[j]) & (time_s <= Forehand_stop_time_list[j]))

                if len(high_time_indexes[0]) > 0:
                    target_time_s_high = np.linspace(Forehand_start_time_list[j], Forehand_stop_time_list[j],
                                        num=segment_length,
                                        endpoint=True)

                    fn_interpolate = interpolate.interp1d(
                        time_s,  # x values
                        data,  # y values
                        axis=0,  # axis of the data along which to interpolate
                        kind='slinear',  # interpolation method, such as 'linear', 'zero', 'nearest', 'quadratic', 'cubic', etc.
                        fill_value='extrapolate'  # how to handle x values outside the original range
                    )

                    data_resampled = fn_interpolate(target_time_s_high)

                    example_matrices_each_file.append(
                        data_resampled.tolist())
                    highNum += 1

                    if len(device_streams_for_features) == device_num:
                        sub_example_label_indexes.append(1)
                        sub_example_labels.append('Forehand Clear')
                        sub_example_subject_ids.append(subject_id)
                        sub_example_skill_level.append(Forehand_skill_level_list[j])
                        Num_clear += 1
                        # sub_example_score.append(Forehand_score_list[j])
                        sub_example_score_annot_3_hori.append(Forehand_score_annot_3_hori_list[j])
                        sub_example_score_annot_3_ver.append(Forehand_score_annot_3_ver_list[j])
                        sub_example_score_annot_4.append(Forehand_score_annot_4_list[j])
                        sub_example_score_annot_5.append(Forehand_score_annot_5_list[j])
                for m in range(len(high_time_indexes[0])):
                    label_indexes[high_time_indexes[0][m]] = 1

            for j in range(len(Backhand_start_time_list)):
                Backhand_time_list.append(Backhand_stop_time_list[j] - Backhand_start_time_list[j])
                back_time_indexes = np.where(
                    (time_s >= Backhand_start_time_list[j]) & (time_s <= Backhand_stop_time_list[j]))
                if len(back_time_indexes[0]) > 0:

                    target_time_s_back = np.linspace(Backhand_start_time_list[j], Backhand_stop_time_list[j],
                                        num=segment_length,
                                        endpoint=True)

                    fn_interpolate = interpolate.interp1d(
                        time_s,  # x values
                        data,  # y values
                        axis=0,  # axis of the data along which to interpolate
                        kind='slinear',  # interpolation method, such as 'linear', 'zero', 'nearest', 'quadratic', 'cubic', etc.
                        fill_value='extrapolate'  # how to handle x values outside the original range
                    )

                    data_resampled = fn_interpolate(target_time_s_back)

                    example_matrices_each_file.append(
                        data_resampled.tolist())

                    backNum += 1
                    if len(device_streams_for_features) == device_num:
                        sub_example_label_indexes.append(0)
                        sub_example_labels.append('Backhand Driving')
                        sub_example_subject_ids.append(subject_id)
                        sub_example_skill_level.append(Backhand_skill_level_list[j])
                        Num_drive += 1
                        # sub_example_score.append(Backhand_score_list[j])
                        sub_example_score_annot_3_hori.append(Backhand_score_annot_3_hori_list[j])
                        sub_example_score_annot_3_ver.append(Backhand_score_annot_3_ver_list[j])
                        sub_example_score_annot_4.append(Backhand_score_annot_4_list[j])
                        sub_example_score_annot_5.append(Backhand_score_annot_5_list[j])
                for m in range(len(back_time_indexes[0])):
                    label_indexes[back_time_indexes[0][m]] = 0


            # Save the Baseline Data
            for j in range(len(NoActivity_start_time_list)):
                NoActivity_time_list.append(NoActivity_stop_time_list[j] - NoActivity_start_time_list[j])
                no_time_indexes = np.where(
                    (time_s >= NoActivity_start_time_list[j]) & (time_s <= NoActivity_stop_time_list[j]))
                if len(no_time_indexes[0]) > 0:

                    target_time_s_no = np.linspace(NoActivity_start_time_list[j], NoActivity_stop_time_list[j],
                                        num=segment_length,
                                        endpoint=True)

                    fn_interpolate = interpolate.interp1d(
                        time_s,  # x values
                        data,  # y values
                        axis=0,  # axis of the data along which to interpolate
                        kind='slinear',  # interpolation method, such as 'linear', 'zero', 'nearest', 'quadratic', 'cubic', etc.
                        fill_value='extrapolate'  # how to handle x values outside the original range
                    )

                    data_resampled = fn_interpolate(target_time_s_no)

                    example_matrices_each_file.append(
                        data_resampled.tolist())

                    baseNum += 1
                    if len(device_streams_for_features) == device_num:
                        sub_example_label_indexes.append(2)
                        sub_example_labels.append('Baseline')
                        sub_example_subject_ids.append(subject_id)
                        sub_example_skill_level.append(-1)
                        Num_base += 1
                        # sub_example_score.append(Backhand_score_list[j])
                        sub_example_score_annot_3_hori.append(-1)
                        sub_example_score_annot_3_ver.append(-1)
                        sub_example_score_annot_4.append(-1)
                        sub_example_score_annot_5.append(-1)
                for m in range(len(no_time_indexes[0])):
                    label_indexes[no_time_indexes[0][m]] = 2


            if device_name == "eye-gaze":
                example_matrices_device_eye_gaze.append(example_matrices_each_file)
            elif device_name == "gforce-lowerarm-emg":
                example_matrices_device_gforce_lowerarm_emg.append(example_matrices_each_file)
            elif device_name == "gforce-upperarm-emg":
                example_matrices_device_gforce_upperarm_emg.append(example_matrices_each_file)
            elif device_name == "cgx-aim-leg-emg":
                example_matrices_device_cgx_aim_emg.append(example_matrices_each_file)
            elif device_name == "moticon-insole" and stream_name == 'left-pressure':
                example_matrices_device_moticon_insole_left_pressure.append(example_matrices_each_file)
            elif device_name == "moticon-insole" and stream_name == 'right-pressure':
                example_matrices_device_moticon_insole_right_pressure.append(example_matrices_each_file)
            elif device_name == "moticon-insole" and stream_name == 'cop':
                example_matrices_device_moticon_insole_cop.append(example_matrices_each_file)
            elif device_name == "pns-joint":
                example_matrices_device_pns_joint_Euler.append(example_matrices_each_file)

            device_num += 1
            # print("highNum")
            # print(highNum)
            # print("backNum")
            # print(backNum)
            # print("baseNum")
            # print(baseNum)
            # print("totalNum")
            # print(highNum + backNum + baseNum)

        device_array1 = np.squeeze(np.array(example_matrices_device_eye_gaze), axis=0)
        device_array2 = np.squeeze(np.array(example_matrices_device_gforce_lowerarm_emg), axis=0)
        device_array3 = np.squeeze(np.array(example_matrices_device_gforce_upperarm_emg), axis=0)
        device_array4 = np.squeeze(np.array(example_matrices_device_cgx_aim_emg), axis=0)
        device_array5 = np.squeeze(np.array(example_matrices_device_moticon_insole_left_pressure), axis=0)
        device_array6 = np.squeeze(np.array(example_matrices_device_moticon_insole_right_pressure), axis=0)
        device_array7 = np.squeeze(np.array(example_matrices_device_moticon_insole_cop), axis=0)
        device_array8 = np.squeeze(np.array(example_matrices_device_pns_joint_Euler), axis=0)

        print("Feature shape of each device")
        print(device_array1.shape)
        print(device_array2.shape)
        print(device_array3.shape)
        print(device_array4.shape)
        print(device_array5.shape)
        print(device_array6.shape)
        print(device_array7.shape)
        print(device_array8.shape)

        if len(device_array1) == 0:
            continue

        if all(array.shape[0] == device_array1.shape[0] for array in [device_array2, device_array3, device_array4, device_array5, device_array6, device_array7, device_array8]):
            combined_array = np.concatenate((device_array1, device_array2, device_array3, device_array4, device_array5, device_array6, device_array7, device_array8), axis=2)

            print(combined_array.shape)
            print(np.array(sub_example_label_indexes).shape)
            print(np.array(sub_example_labels).shape)
            print(np.array(sub_example_subject_ids).shape)
            example_label_indexes.extend(sub_example_label_indexes)
            example_labels.extend(sub_example_labels)
            example_subject_ids.extend(sub_example_subject_ids)
            example_skill_level.extend(sub_example_skill_level)
            example_score_annot_3_hori.extend(sub_example_score_annot_3_hori)
            example_score_annot_3_ver.extend(sub_example_score_annot_3_ver)
            example_score_annot_4.extend(sub_example_score_annot_4)
            example_score_annot_5.extend(sub_example_score_annot_5)
        else:
            min_ = min(device_array1.shape[0], device_array2.shape[0], device_array3.shape[0], device_array4.shape[0], device_array5.shape[0], device_array6.shape[0], device_array7.shape[0], device_array8.shape[0])
            print(min_)
            example_label_indexes.extend(sub_example_label_indexes[:min_])
            example_labels.extend(sub_example_labels[:min_])
            example_subject_ids.extend(sub_example_subject_ids[:min_])
            combined_array = np.concatenate((device_array1[:min_, :, :], device_array2[:min_, :, :], device_array3[:min_, :, :], device_array4[:min_, :, :],
                                             device_array5[:min_, :, :], device_array6[:min_, :, :], device_array7[:min_, :, :], device_array8[:min_, :, :]), axis=2)
            example_skill_level.extend(sub_example_skill_level[:min_])
            example_score_annot_3_hori.extend(sub_example_score_annot_3_hori[:min_])
            example_score_annot_3_ver.extend(sub_example_score_annot_3_ver[:min_])
            example_score_annot_4.extend(sub_example_score_annot_4[:min_])
            example_score_annot_5.extend(sub_example_score_annot_5[:min_])
            print(combined_array.shape)
        # combined_array_squeezd = np.squeeze(combined_array, axis=0)
        example_matrices_list.append(combined_array)

Resampling data for subject Sub00
Total Labeling Number : 638
Saved Highclear Labeling Number : 166 166
Saved Backhand Labeling Number : 154 154
Saved NoActivity Labeling Number : 318 318
Save Forehand Score annot 3 Hori Number : 166
Save Forehand Score annot 3 Ver Number : 166
Save Forehand Score annot 4 Number : 166
Save Forehand Score annot 5 Number : 166
Save Backhand Score annot 3 Hori Number : 154
Save Backhand Score annot 3 Ver Number : 154
Save Backhand Score annot 4 Number : 154
Save Backhand Score annot 5 Number : 154
Device Name : eye-gaze
Device Name : gforce-lowerarm-emg
Device Name : gforce-upperarm-emg
Device Name : cgx-aim-leg-emg
Device Name : moticon-insole
Device Name : moticon-insole
Device Name : moticon-insole
Device Name : pns-joint
Feature shape of each device
(307, 250, 2)
(298, 250, 8)
(307, 250, 8)
(307, 250, 4)
(307, 250, 16)
(307, 250, 16)
(307, 250, 4)
(307, 250, 63)
298
(298, 250, 121)
Total Labeling Number : 638
Saved Highclear Labeling Number : 166 166


In [ ]:
def sanitize_for_hdf5(data):
    """
    Convierte cualquier lista/array/mixto en un formato compatible con h5py:
    - Listas mixtas -> strings
    - np.nan o None -> 'NaN'
    - Sublistas -> las convierte a texto
    """
    cleaned = []
    for x in data:
        # Si es None o NaN -> representarlo como texto
        if x is None or (isinstance(x, float) and np.isnan(x)):
            cleaned.append("NaN")
        # Si es lista, tupla o array, convertirla a texto
        elif isinstance(x, (list, tuple, np.ndarray)):
            cleaned.append(str(x))
        # Caso normal: convertir a texto
        else:
            cleaned.append(str(x))
    return np.array(cleaned, dtype=h5py.string_dtype(encoding="utf-8"))

In [ ]:
example_matrices = np.concatenate([arr for arr in example_matrices_list], axis=0)

def prepare_data_for_hdf5(data):
    """Convierte datos a formato compatible con HDF5"""
    arr = np.array(data)
    # Si es string Unicode u Object, convertir a bytes
    if arr.dtype.kind in ('U', 'O'):
        return arr.astype('S')
    return arr

print('total feature shape')
print(example_matrices.shape)
print(np.array(example_labels).shape)
print(np.array(example_label_indexes).shape)
print(np.array(example_subject_ids).shape)

if output_filepath is not None:
    with h5py.File(output_filepath, 'w') as hdf_file:
        metadata = OrderedDict()
        metadata['output_dir'] = output_dir
        metadata['data_root_dir'] = data_root_dir
        metadata['data_folders_bySubject'] = data_folders_bySubject
        metadata['activities_to_classify'] = activities_to_classify
        metadata['device_streams_for_features'] = device_streams_for_features
        metadata['resampled_Fs'] = resampled_Fs
        metadata['segment_length'] = segment_length
        metadata['segment_duration_s'] = segment_duration_s
        metadata['filter_cutoff_emg_Hz'] = filter_cutoff_emg_Hz
        metadata['filter_cutoff_pressure_Hz'] = filter_cutoff_pressure_Hz
        metadata['filter_cutoff_gaze_Hz'] = filter_cutoff_gaze_Hz

        metadata = dict_utils.convert_dict_values_to_str(metadata, preserve_nested_dicts=False)

        hdf_file.create_dataset('example_labels', data=example_labels)
        hdf_file.create_dataset('example_label_indexes', data=example_label_indexes)
        hdf_file.create_dataset('example_matrices', data=example_matrices)
        hdf_file.create_dataset('example_subject_ids', data=example_subject_ids)
        hdf_file.create_dataset('example_skill_level', data=example_skill_level)
        # hdf_file.create_dataset('example_score', data=example_score)
        # hdf_file.create_dataset('example_score_annot_3_hori', data=example_score_annot_3_hori)
        # hdf_file.create_dataset('example_score_annot_3_ver', data=example_score_annot_3_ver)
        # hdf_file.create_dataset('example_score_annot_4', data=example_score_annot_4)
        # hdf_file.create_dataset('example_score_annot_5', data=example_score_annot_5)
        hdf_file.attrs.update(metadata)

        print()
        print('Saved processed data to', output_filepath)
        print()

# if output_filepath is not None:
#     with h5py.File(output_filepath, 'w') as hdf_file:
#         metadata = OrderedDict()
#         metadata['output_dir'] = output_dir
#         metadata['data_root_dir'] = data_root_dir
#         metadata['data_folders_bySubject'] = data_folders_bySubject
#         metadata['activities_to_classify'] = activities_to_classify
#         metadata['device_streams_for_features'] = device_streams_for_features
#         metadata['resampled_Fs'] = resampled_Fs
#         metadata['segment_length'] = segment_length
#         metadata['segment_duration_s'] = segment_duration_s
#         metadata['filter_cutoff_emg_Hz'] = filter_cutoff_emg_Hz
#         metadata['filter_cutoff_pressure_Hz'] = filter_cutoff_pressure_Hz
#         metadata['filter_cutoff_gaze_Hz'] = filter_cutoff_gaze_Hz

#         metadata = dict_utils.convert_dict_values_to_str(metadata, preserve_nested_dicts=True)

#         hdf_file.create_dataset('example_labels', data=example_labels)
#         hdf_file.create_dataset('example_label_indexes', data=example_label_indexes)
#         hdf_file.create_dataset('example_matrices', data=example_matrices)
#         hdf_file.create_dataset('example_subject_ids', data=example_subject_ids)
#         #hdf_file.create_dataset('example_skill_level', data=example_skill_level)
#         hdf_file.create_dataset('example_skill_level', data=sanitize_for_hdf5(example_skill_level))
#         # hdf_file.create_dataset('example_score', data=example_score)
#         hdf_file.create_dataset('example_score_annot_3_hori', data=example_score_annot_3_hori)
#         hdf_file.create_dataset('example_score_annot_3_ver', data=example_score_annot_3_ver)
#         hdf_file.create_dataset('example_score_annot_4', data=example_score_annot_4)
#         hdf_file.create_dataset('example_score_annot_5', data=example_score_annot_5)

#         import json

#         # Convertir metadata a texto JSON legible
#         try:
#             hdf_file.attrs['metadata_json'] = json.dumps(metadata, indent=2, default=str)
#         except Exception as e:
#             print("Error serializando metadata a JSON:", e)
#             # Guardar versión simplificada (solo claves principales)
#             safe_meta = {k: str(v) for k, v in metadata.items()}
#             hdf_file.attrs['metadata_str'] = json.dumps(safe_meta)

#         print()
#         print('Saved processed data to', output_filepath)
#         print()

total feature shape
(10929, 250, 121)
(10929,)
(10929,)
(10929,)

Saved processed data to /content/drive/MyDrive/TFM_2/data_processed/data_processed_allStreams_60z_18subj_allActs_mulltiple_obj.hdf5

